In [1]:
%load_ext autoreload
%autoreload 2

import daimler
import os

In [5]:
basepath="/media/data/datasets/pedestrian/daimler_mono/DaimlerBenchmark/"
print("Loading dataset...")
train_pedestrian,train_non_pedestrian,metadata,object_types=daimler.get_dataset(basepath)
print("Loaded")

Loading dataset...
Loaded


In [40]:
# tr
import numpy as np
from hog import calculate_hog,calculate_descriptor_windows


def crop_images_uniform(images,crop_grid_size,crop_size):
    n=len(images)
    ch,cw=crop_size
    gh,gw=crop_grid_size
    crops_per_image=gh*gw
    crops=np.zeros((n*crops_per_image,ch,cw))
    j=0
    for i in range(n):
        image=images[i]
        h,w=image.shape
        assert ch+gh<=h and cw+gw<=w
        adjusted_h=h-ch
        adjusted_w=w-cw
        h_points=np.linspace(0,adjusted_h,gh, dtype = int, endpoint=False)
        w_points=np.linspace(0,adjusted_w,gw, dtype = int, endpoint=False)
        assert len(h_points) == gh
        assert len(w_points) == gw
        for t in h_points:
            for l in w_points:
                crops[j,:,:]=image[t:t+ch,l:l+cw]
                j+=1
    return crops
    
def hogs_from_images(images):
            
    first_image=images[0]
    print("image shape %s" % str(first_image.shape))

    hog=calculate_hog(first_image,cell_size=6)
    print("resulting hog size = %d" % len(hog))
    n=len(images)
    hogs=np.zeros((n,hog.shape[0]))

    print("Generating hogs for %d images..." % n)
    for i in range(n):
        hogs[i,:]=calculate_hog(images[i],cell_size=6)
        if i % (n//10)==0:
            print("  %f \%.." % (i/n*100))
    print("Done")    
    return hogs


                   


In [ ]:
print("Generating hogs for pedestrians")
train_pedestrian_hogs=hogs_from_images(train_pedestrian)

In [41]:
image_size=train_pedestrian[0].shape
crop_size=image_size
crop_grid_size=(2,2)
crops_per_image=crop_grid_size[0]*crop_grid_size[1]
print("Generating %d crops of size %s per image " % (crops_per_image, str(image_size)))
train_non_pedestrian_crops=crop_images_uniform(train_non_pedestrian,crop_grid_size,crop_size)
print("Generating hogs for non pedestrians")
train_non_pedestrian_hogs=hogs_from_images(train_non_pedestrian_crops)

image shape (96, 48)
resulting hog size = 6804
Generating hogs for 40464 images...
  0.000000..
  9.999011..


KeyboardInterrupt: 

In [ ]:

print("Storing hogs..")

print("Done")

In [ ]:
# descriptor_function= lambda image: calculate_hog(image)
# hog_windows=calculate_descriptor_windows(image_rgb,descriptor_function)